# Test cython pytable shortest path

https://stackoverflow.com/questions/25917593/how-to-declare-2d-c-arrays-dynamically-in-cython

In [12]:
%load_ext autoreload
%autoreload 2

import pyximport
import os
import sys
import importlib
import cProfile
import pandas as pd
import numpy as np
sys.path.insert(0, os.path.abspath('../../'))
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug('This will get logged')

import tables as tb
from tables import File, Table
from tables.file import _checkfilters

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

DEBUG:root:This will get logged


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from converter.input_converter import Converter
from converter.import_Belenguer_format import convert_file
import converter.py_data_write as py_data_write
import converter.shortest_paths as shortest_paths
from converter import load_instance
from converter import Network
from converter import ShortestPath

In [14]:
file = 'temp/Lpr_IF-c-05' 

In [18]:
instance_info = convert_file('{}.txt'.format(file))
instance_df = instance_info[['arc_u', 'arc_v', 'arc_cost']]
instance_df['arc_id'] = 'test_id'
network = Network(instance_df, second_key = 'arc_id')
network.create_pytable('{}_fixed_test_play.h5'.format(file), overwrite=True)
sp_calc = ShortestPath()
sp_calc.load_from_pytable('{}_fixed_test_play.h5'.format(file))

'Converting temp/Lpr_IF-c-05.txt...'


<ipython-input-18-a2066b9d2a2a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instance_df['arc_id'] = 'test_id'


Press 'y' to continue y


INFO:root:Creating a network pytable at temp/Lpr_IF-c-05_fixed_test_play.h5
INFO:root:Writing successor arcs
INFO:root:Pytable successfully created.
INFO:root:temp/Lpr_IF-c-05_fixed_test_play.h5 (File) ''
Last modif.: 'Tue Jul 21 05:12:00 2020'
Object Tree: 
/ (RootGroup) ''
/arc_info (Group) 'Network arc info (all are index based)'
/arc_info/arc_cost (Array(1231,)) 'Cost of traversing arc'
/arc_info/arc_index (Array(1231,)) 'Arc index (original position starting at 0)'
/arc_info/arc_pair (Array(1231,)) "Arc pair '{arc_u}-{arc_v}', can be used as a  secondary key"
/arc_info/arc_successor_index_list (VLArray(1231,), shuffle, zlib(1)) 'Successor arc indices of arc u-v'
/arc_info/arc_u (Array(1231,)) 'Start node of arc'
/arc_info/arc_v (Array(1231,)) 'End node of arc'
/arc_info/secondary_key (Array(1231,)) 'Secondary key to trace the network back to its origins'

INFO:root:Loading temp/Lpr_IF-c-05_fixed_test_play.h5
INFO:root:temp/Lpr_IF-c-05_fixed_test_play.h5 (File) ''
Last modif.: 'Tue

In [19]:
sp_calc.calc_shortest_path_pytable()

INFO:root:Writing shortest path info to temp/Lpr_IF-c-05_fixed_test_play.h5
INFO:root:Shortest path info added.
INFO:root:Starting shortest path calculations (3 procedures)
INFO:root:1 of 3: Initialise matrices
INFO:root:2 of 3: Calculate shortest paths
DEBUG:root:2 of 3: SP calc 1 out of 1231
DEBUG:root:2 of 3: SP calc 2 out of 1231
DEBUG:root:2 of 3: SP calc 3 out of 1231
DEBUG:root:2 of 3: SP calc 4 out of 1231
DEBUG:root:2 of 3: SP calc 5 out of 1231
DEBUG:root:2 of 3: SP calc 6 out of 1231
DEBUG:root:2 of 3: SP calc 7 out of 1231
DEBUG:root:2 of 3: SP calc 8 out of 1231
DEBUG:root:2 of 3: SP calc 9 out of 1231
DEBUG:root:2 of 3: SP calc 10 out of 1231
DEBUG:root:2 of 3: SP calc 11 out of 1231
DEBUG:root:2 of 3: SP calc 12 out of 1231
DEBUG:root:2 of 3: SP calc 13 out of 1231
DEBUG:root:2 of 3: SP calc 14 out of 1231
DEBUG:root:2 of 3: SP calc 15 out of 1231
DEBUG:root:2 of 3: SP calc 16 out of 1231
DEBUG:root:2 of 3: SP calc 17 out of 1231
DEBUG:root:2 of 3: SP calc 18 out of 1231

In [21]:
with tb.open_file('{}_fixed_test_play.h5'.format(file), 'r') as h5file:
    arc_info = h5file.root.arc_info
    sp_info = h5file.root.shortest_path_info
    cost_matrix = sp_info.cost_matrix[:, :]
    cost_matrix
    predecessor_matrix
    sp_info.calculation_time[0]
    arc_info.secondary_key[:]
    print(h5file)

array([[  0, 256, 325, ..., 296, 296, 305],
       [252,   0, 198, ..., 227, 227, 218],
       [309, 189,   0, ...,  40,  40,  20],
       ...,
       [263, 211,  63, ...,   0,  23,  43],
       [296, 203,  20, ...,  20,   0,   0],
       [286, 223,  40, ...,   0,   0,   0]], dtype=int32)

array([[   0,  851,  453, ...,  437,  437, 1197],
       [ 870,    1,  456, ...,  437,  437,  454],
       [ 290,  278,    2, ..., 1230, 1230,  454],
       ...,
       [ 290,  278,  453, ...,    0, 1227, 1229],
       [ 290,  278,  453, ..., 1230,    0, 1229],
       [ 290,  278,  453, ..., 1230, 1230,    0]], dtype=int32)

3.480492013999992

array([b'test_id', b'test_id', b'test_id', ..., b'test_id', b'test_id',
       b'test_id'], dtype='|S7')

temp/Lpr_IF-c-05_fixed_test_play.h5 (File) ''
Last modif.: 'Tue Jul 21 05:12:12 2020'
Object Tree: 
/ (RootGroup) ''
/arc_info (Group) 'Network arc info (all are index based)'
/arc_info/arc_cost (Array(1231,)) 'Cost of traversing arc'
/arc_info/arc_index (Array(1231,)) 'Arc index (original position starting at 0)'
/arc_info/arc_pair (Array(1231,)) "Arc pair '{arc_u}-{arc_v}', can be used as a  secondary key"
/arc_info/arc_successor_index_list (VLArray(1231,), shuffle, zlib(1)) 'Successor arc indices of arc u-v'
/arc_info/arc_u (Array(1231,)) 'Start node of arc'
/arc_info/arc_v (Array(1231,)) 'End node of arc'
/arc_info/secondary_key (Array(1231,)) 'Secondary key to trace the network back to its origins'
/shortest_path_info (Group) 'Shortest path info for network'
/shortest_path_info/calculation_time (Array(1,)) 'Shortest path calculation time (seconds)'
/shortest_path_info/cost_matrix (CArray(1231, 1231), shuffle, zlib(1)) 'shortest path cost matrix between all arc indices'
/shortest_p

In [7]:
info = load_instance('{}.txt'.format(file), name_tuple=False, overwrite=True)

'4 files will be overwritten'
['`temp/Lpr_IF-c-05_info_lists_pickled.dat` exists',
 '`temp/Lpr_IF-c-05_problem_info.dat` exists',
 '`temp/Lpr_IF-c-05_sp_data_full.dat` exists',
 '`temp/Lpr_IF-c-05_nn_list.dat` exists']
'Converting temp/Lpr_IF-c-05.txt...'

Starting shortest path calculations (3 procedures)

    1 of 3: Initialise matrices
    2 of 3: Calculate shortest paths

        2 of 3: SP calc 1 out of 1231
        2 of 3: SP calc 2 out of 1231
        2 of 3: SP calc 3 out of 1231
        2 of 3: SP calc 4 out of 1231
        2 of 3: SP calc 5 out of 1231
        2 of 3: SP calc 6 out of 1231
        2 of 3: SP calc 7 out of 1231
        2 of 3: SP calc 8 out of 1231
        2 of 3: SP calc 9 out of 1231
        2 of 3: SP calc 10 out of 1231
        2 of 3: SP calc 11 out of 1231
        2 of 3: SP calc 12 out of 1231
        2 of 3: SP calc 13 out of 1231
        2 of 3: SP calc 14 out of 1231
        2 of 3: SP calc 15 out of 1231
        2 of 3: SP calc 16 out of 1231
      

In [8]:
d_test = np.array(info.d_full)
p_test = np.array(info.p_full)

In [9]:
np.array_equal(d_test, cost_matrix)
np.array_equal(p_test, predecessor_matrix)

True

True